## Setup


In [ ]:
cd /root/Development/diffusers/examples/dreambooth

python3 -m venv venv
source venv/bin/activate

pip install notebook

git clone https://github.com/huggingface/diffusers
cd diffusers
pip install -e .
cd ..

pip install -r requirements.txt

pip install safetensors

accelerate config


In [ ]:
git lfs install

git clone https://huggingface.co/CompVis/stable-diffusion-v1-4
git clone https://huggingface.co/runwayml/stable-diffusion-v1-5
git clone https://huggingface.co/stabilityai/stable-diffusion-2-1-base

In [ ]:
git clone https://github.com/djbielejeski/Stable-Diffusion-Regularization-Images-person_ddim.git
# git clone https://github.com/djbielejeski/Stable-Diffusion-Regularization-Images-man_1_ddim_step.git
git clone https://github.com/djbielejeski/Stable-Diffusion-Regularization-Images-man_euler
git clone https://github.com/djbielejeski/Stable-Diffusion-Regularization-Images-person-photographs

In [ ]:
from huggingface_hub import snapshot_download

local_dir = "./dog"
snapshot_download(
    "diffusers/dog-example",
    local_dir=local_dir,
    repo_type="dataset",
    ignore_patterns=".gitattributes",
)

# Init


In [ ]:
!accelerate env

In [ ]:
ls

## Parameters


In [ ]:
# MODEL_NAME = "CompVis/stable-diffusion-v1-4"
# MODEL_NAME = "runwayml/stable-diffusion-v1-5"
# MODEL_NAME = "stabilityai/stable-diffusion-2-1-base"
# MODEL_NAME = "stabilityai/stable-diffusion-2-1"
# MODEL_NAME = "BAAI/AltDiffusion-m9"
# MODEL_NAME = "BAAI/AltDiffusion"

MODEL_NAME = "stable-diffusion-v1-5"
# MODEL_NAME = "stable-diffusion-2-1-base"
MODEL_PATH = f"training_models/{MODEL_NAME}"

# INSTANCE_NAME = "dog"
# INSTANCE_NAME = "rabbit_toy"
# INSTANCE_NAME = "gabrieltorcat"
# INSTANCE_NAME = "gabrieltorcat2"
INSTANCE_NAME = "gabrieltorcat_512"
INSTANCE_DIR = f"training_images/{INSTANCE_NAME}"
# CLASS_DIR = "dog"
# CLASS_DIR = "toy"
CLASS_DIR = "man"
# CLASS_DIR = "person"
# CLASS_DIR = "Stable-Diffusion-Regularization-Images-man_1_ddim_step/man_1_ddim_step"
# CLASS_DIR = "Stable-Diffusion-Regularization-Images-person_ddim/person_ddim"
CLASS_DIR = f"regularization_images/{MODEL_NAME}/{CLASS_DIR}"
OUTPUT_DIR = f"outputs/{MODEL_NAME}/{INSTANCE_NAME}"

# PROMPT_TOKEN = "sks"
# PROMPT_TOKEN = "dbDog"
# PROMPT_TOKEN = "dbRabbit"
PROMPT_TOKEN = "gabrieltorcat"
# CLASS_TOKEN = "dog"
# CLASS_TOKEN = "toy"
CLASS_TOKEN = "man"
# CLASS_TOKEN = "person"
INSTANCE_PROMPT = f"a photo of {PROMPT_TOKEN} {CLASS_TOKEN}"
# VALIDATION_PROMPT = f"an oil painting of {PROMPT_TOKEN} {CLASS_TOKEN} sitting next to a wooden window reading a book, by Vincent Van Gogh"
VALIDATION_PROMPT = (
    # f"an oil painting of {PROMPT_TOKEN} {CLASS_TOKEN}, by Vincent Van Gogh"
    f"a photo of {PROMPT_TOKEN} {CLASS_TOKEN} riding a bike in New York city"
)
CLASS_PROMPT = f"a photo of {CLASS_TOKEN}"

RESOLUTION = 512
# RESOLUTION = 768
TRAIN_BATCH_SIZE = 1
# TRAIN_BATCH_SIZE = 2

GRADIENT_ACCUMULATION_STEPS = 1
# GRADIENT_ACCUMULATION_STEPS = 2

# LEARNING_RATE = 5e-6
# LEARNING_RATE = 2e-6
# LEARNING_RATE = 1e-6
# LEARNING_RATE = 9e-7
LEARNING_RATE = 8e-7

# MAX_TRAIN_STEPS = 400
# MAX_TRAIN_STEPS = 800
# MAX_TRAIN_STEPS = 1200
MAX_TRAIN_STEPS = 1600

# NUM_CLASS_IMAGES = 50
# NUM_CLASS_IMAGES = 100
# NUM_CLASS_IMAGES = 300
NUM_CLASS_IMAGES = 500
# NUM_CLASS_IMAGES = 1500
SAMPLE_BATCH_SIZE = 1
# SAMPLE_BATCH_SIZE = 2

CHECKPOINTING_STEPS = 200
# CHECKPOINTING_STEPS = 300
# CHECKPOINTING_STEPS = 400

HUB_TOKEN = ""

# NUM_CPU_THREADS_PER_PROCESS = 16
NUM_CPU_THREADS_PER_PROCESS = 20

## Training


In [ ]:

accelerate launch \
  
  --num_cpu_threads_per_process=$NUM_CPU_THREADS_PER_PROCESS \
  
  train_dreambooth.py \
  
  --pretrained_model_name_or_path="$MODEL_PATH" \

  --train_text_encoder \

# bitsandbytes
#   --use_8bit_adam \
#   --gradient_checkpointing \

# xformers
#   --enable_xformers_memory_efficient_attention \
#   --set_grads_to_none \

  --instance_data_dir="$INSTANCE_DIR" \
  --class_data_dir="$CLASS_DIR" \
  --output_dir="$OUTPUT_DIR" \

  --class_prompt="$CLASS_PROMPT" \
  --instance_prompt="$INSTANCE_PROMPT" \
  --validation_prompt="$VALIDATION_PROMPT" \

  --resolution=$RESOLUTION \
  --train_batch_size=$TRAIN_BATCH_SIZE \

  --gradient_accumulation_steps=$GRADIENT_ACCUMULATION_STEPS \
  --learning_rate=$LEARNING_RATE \

  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --max_train_steps=$MAX_TRAIN_STEPS \

  --with_prior_preservation \
  --prior_loss_weight=1.0 \
  --num_class_images=$NUM_CLASS_IMAGES \
  --sample_batch_size=$SAMPLE_BATCH_SIZE \

  --checkpointing_steps=$CHECKPOINTING_STEPS \
  # --resume_from_checkpoint="checkpoint-1500" \
  --resume_from_checkpoint="latest" \

  # --push_to_hub \
  # --hub_token=$HUB_TOKEN \



In [ ]:
!accelerate launch \
  --num_cpu_threads_per_process=$NUM_CPU_THREADS_PER_PROCESS \
  train_dreambooth.py \
  --pretrained_model_name_or_path="$MODEL_PATH" \
  --train_text_encoder \
  --instance_data_dir="$INSTANCE_DIR" \
  --class_data_dir="$CLASS_DIR" \
  --output_dir="$OUTPUT_DIR" \
  --class_prompt="$CLASS_PROMPT" \
  --instance_prompt="$INSTANCE_PROMPT" \
  --validation_prompt="$VALIDATION_PROMPT" \
  --resolution=$RESOLUTION \
  --train_batch_size=$TRAIN_BATCH_SIZE \
  --gradient_accumulation_steps=$GRADIENT_ACCUMULATION_STEPS \
  --learning_rate=$LEARNING_RATE \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --max_train_steps=$MAX_TRAIN_STEPS \
  --with_prior_preservation \
  --prior_loss_weight=1.0 \
  --num_class_images=$NUM_CLASS_IMAGES \
  --sample_batch_size=$SAMPLE_BATCH_SIZE \
  --checkpointing_steps=$CHECKPOINTING_STEPS \
  --resume_from_checkpoint="latest"


### convert to stable diffusion format


In [ ]:
!python ../../scripts/convert_diffusers_to_original_stable_diffusion.py \
    --model_path="$OUTPUT_DIR" \
    --checkpoint_path="$OUTPUT_DIR/sd_weights.ckpt" \
    --half \
    # --use_safetensors \

## Tensorboard


In [ ]:
%tensorboard --logdir OUTPUT_DIR + "/logs/dreambooth"


In [ ]:
!tensorboard --logdir $OUTPUT_DIR"/logs/dreambooth"

## Inference


In [ ]:
import os
import datetime
from diffusers import (
    DiffusionPipeline,
    UNet2DConditionModel,
    StableDiffusionPipeline,
    DDIMScheduler,
    DPMSolverMultistepScheduler,
)
from transformers import CLIPTextModel
import torch
from IPython.display import display

prompts = [
    INSTANCE_PROMPT,
    VALIDATION_PROMPT,
    INSTANCE_PROMPT + " on top of mount fuji",
    INSTANCE_PROMPT + " shaking hands with barack obama",
    INSTANCE_PROMPT + " in front of the eiffel tower",
    INSTANCE_PROMPT + " as an astronaut",
    INSTANCE_PROMPT + " as a mecha robot",
    INSTANCE_PROMPT + " shaking hands with emmanuel macron",
    INSTANCE_PROMPT + " in a ramen bowl",
    INSTANCE_PROMPT + " as neo fighting morpheus from the movie the matrix",
    INSTANCE_PROMPT + " doing a handstand",
    INSTANCE_PROMPT + " as a fighter jet pilot",
    INSTANCE_PROMPT + " by the ocean",
    INSTANCE_PROMPT + " holding the football world cup",
    INSTANCE_PROMPT + " as mad max from mad max fury road",
    INSTANCE_PROMPT + " as the joker from the dark night",
    INSTANCE_PROMPT + " as the terminator",
    INSTANCE_PROMPT + " as a Na'vi from the movie Avatar",
    INSTANCE_PROMPT + " as a hobbit",
    INSTANCE_PROMPT + " as an olympic athlete",
    INSTANCE_PROMPT + " as a cowboy",
    INSTANCE_PROMPT + " as a pirate",
]

CHECKPOINTS = [
    200,
    400,
    600,
    800,
    1000,
    1200,
    1400,
    1600,
    None,
]

# Load the pipeline with the same arguments (model, revision) that were used for training
model_id = MODEL_PATH

print(f"Model = {model_id}")

for checkpoint in CHECKPOINTS:
    print(f"Checkpoint = {checkpoint}")

    if checkpoint:
        image_dir = (
            OUTPUT_DIR + "/test_output_images" + "/checkpoint-" + str(checkpoint)
        )

        if image_dir is not None:
            os.makedirs(image_dir, exist_ok=True)

        unet = UNet2DConditionModel.from_pretrained(
            OUTPUT_DIR + "/checkpoint-" + str(checkpoint) + "/unet"
        )

        text_encoder = CLIPTextModel.from_pretrained(
            OUTPUT_DIR + "/checkpoint-" + str(checkpoint) + "/text_encoder"
        )

        pipeline = DiffusionPipeline.from_pretrained(
            model_id, unet=unet, text_encoder=text_encoder, torch_dtype=torch.float32
        )

    else:
        model_id = OUTPUT_DIR

        print(f"Model = {model_id}")

        image_dir = OUTPUT_DIR + "/test_output_images"

        if image_dir is not None:
            os.makedirs(image_dir, exist_ok=True)

        pipeline = StableDiffusionPipeline.from_pretrained(
            model_id, torch_dtype=torch.float32
        )

    # pipeline.to("cuda")
    # pipeline.scheduler = DPMSolverMultistepScheduler.from_config(pipeline.scheduler.config)
    pipeline.scheduler = DDIMScheduler.from_config(pipeline.scheduler.config)
    # pipeline = pipeline.to(accelerator.device)
    # pipeline.set_progress_bar_config(disable=True)

    # Perform inference, or save, or push to the hub

    for prompt in prompts:
        print(f"Prompt = {prompt}")

        images = pipeline(
            prompt,
            # negative_prompt="deformed",
            num_inference_steps=20,
            # num_inference_steps=40,
            # num_inference_steps=100,
            guidance_scale=7.5,
            height=512,
            width=512,
            num_images_per_prompt=2,
            # num_images_per_prompt=4,
        ).images

        for img in images:
            now = datetime.datetime.isoformat(datetime.datetime.today())

            display(img)

            img.save(image_dir + "/" + now + ".png")

        # pipeline.save_pretrained("dreambooth-pipeline")

In [ ]:
%reset?????

In [ ]:
# @title Free runtime memory
exit()?????

# OLD


## Gradio


In [ ]:
# @markdown Run Gradio UI for generating images.
import gradio as gr


def inference(
    prompt,
    negative_prompt,
    num_samples,
    height=512,
    width=512,
    num_inference_steps=50,
    guidance_scale=7.5,
):
    with torch.autocast("cuda"), torch.inference_mode():
        return pipe(
            prompt,
            height=int(height),
            width=int(width),
            negative_prompt=negative_prompt,
            num_images_per_prompt=int(num_samples),
            num_inference_steps=int(num_inference_steps),
            guidance_scale=guidance_scale,
            generator=g_cuda,
        ).images


with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column():
            prompt = gr.Textbox(label="Prompt", value="photo of zwx dog in a bucket")
            negative_prompt = gr.Textbox(label="Negative Prompt", value="")
            run = gr.Button(value="Generate")
            with gr.Row():
                num_samples = gr.Number(label="Number of Samples", value=4)
                guidance_scale = gr.Number(label="Guidance Scale", value=7.5)
            with gr.Row():
                height = gr.Number(label="Height", value=512)
                width = gr.Number(label="Width", value=512)
            num_inference_steps = gr.Slider(label="Steps", value=24)
        with gr.Column():
            gallery = gr.Gallery()

    run.click(
        inference,
        inputs=[
            prompt,
            negative_prompt,
            num_samples,
            height,
            width,
            num_inference_steps,
            guidance_scale,
        ],
        outputs=gallery,
    )

demo.launch(debug=True)